<h1>WWOX: Mignot, et al. (2015)</h1>
<p>We will process <a href="https://pubmed.ncbi.nlm.nih.gov/25411445/" target="__blank">Mignot, et al. (2015) WWOX-related encephalopathies: delineation of the phenotypical spectrum and emerging genotype-phenotype correlation</a></p>
<p>According to the authors, the phenotype in four patients carrying two predicted null alleles was characterised by (1) little if any psychomotor acquisitions, poor spontaneous motility and absent eye contact from birth, (2) pharmacoresistant epilepsy starting in the 1st weeks of life, (3) possible retinal degeneration, acquired microcephaly and premature death. This contrasted with the less severe autosomal recessive spinocerebellar ataxia type 12 phenotype due to hypomorphic alleles. </p>

In [1]:
import phenopackets as php
from google.protobuf.json_format import MessageToDict, MessageToJson
from google.protobuf.json_format import Parse, ParseDict
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
import os
import sys
import numpy as np
import pyphetools
from pyphetools.creation import *
from pyphetools.visualization import *
print(f"pyphetools version {pyphetools.__version__}")

pyphetools version 0.6.5


<h2>Importing HPO data</h2>
<p>pyphetools uses the Human Phenotype Ontology (HPO) to encode phenotypic features. The recommended way of doing this is to ingest the hp.json file using HpoParser, which in turn creates an HpoConceptRecognizer object. </p>
<p>The HpoParser can accept a hpo_json_file argument if you want to use a specific file. If the argument is not passed, it will download the latext hp.json file from the HPO GitHub site and store it in a new subdirectory called hpo_data. It will not download the file if the file is already downloaded.</p>

In [2]:
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
PMID = "PMID:25411445"
title = "WWOX-related encephalopathies: delineation of the phenotypical spectrum and emerging genotype-phenotype correlation"
metadata = MetaData(created_by="ORCID:0000-0002-5648-2155", pmid=PMID, pubmed_title=title)
metadata.default_versions_with_hpo(version=hpo_version)

<h2>Importing the supplemental table</h2>

In [3]:
df = pd.read_excel('./input/PMID_25411445.xlsx')
df = df.set_index('Patient').T.reset_index()
df['Age'] = 0
df['patient_id'] = df['index'] 
df.head()

Patient,index,Sex,Age,Mutation,DNA level (NM_016373.2),Second mutation,Protein level (NM_057457),Microcephaly,Profound global developmental delay,Axial hypotonia,...,Hypokinesia,Rigidity,Thin corpus callosum,Delayed myelination,Cerebral atrophy,Seizure,EEG abnormality,Reduced eye contact,Sudden death,patient_id
0,1,F,0,CH for two deletions of several exons,c.366_516del,c.517-?_1056+ ?del,p.[0];[His173_Met352del],Yes,Yes,Yes,...,Yes,Yes,Yes,No,No,Yes,Yes,Yes,No,1
1,2,F,0,CH: deletion of exon 6/ nonsense exon 8,c.1005G>A,c.517-?_605+?del,p.[His173Alafs*67];[Trp335*],Yes,Yes,Yes,...,Yes,Yes,No,Yes,No,Yes,Yes,Yes,Yes,2
2,3,M,0,CH: frameshift exon,c.45_48delGGAC,c.140C>G,p.[Asp16Serfs*63];[Pro47Arg],No,Yes,Yes,...,No,No,No,No,No,Yes,Yes,No,No,3
3,4,F,0,1/missense exon 2,c.45_48delGGAC,c.140C>G,NaN,No,Yes,Yes,...,No,No,No,No,No,Yes,Yes,No,No,4
4,5,F,0,CH: complete deletion/ nonsense exon 8,c.889A>T,c.-366-? *871+?del,p.[0];[Lys297*],No,Yes,Yes,...,Yes,Yes,No,No,Yes,Yes,Yes,Yes,Yes,5


<h2>Column mappers</h2>

In [4]:
generator = SimpleColumnMapperGenerator(df=df, observed='Yes', excluded='No', hpo_cr=hpo_cr)
column_mapper_d = generator.try_mapping_columns()

In [5]:
from IPython.display import display, HTML
display(HTML(generator.to_html()))

Result,Columns
Mapped,Microcephaly; Profound global developmental delay; Axial hypotonia; Cerebellar ataxia; Spasticity; Hypokinesia; Rigidity; Thin corpus callosum; Delayed myelination; Cerebral atrophy; Seizure; EEG abnormality; Reduced eye contact; Sudden death
Unmapped,index; Sex; Age; Mutation; DNA level (NM_016373.2); Second mutation; Protein level (NM_057457); patient_id


<h2>Variant Data</h2>
<p>The variant data (HGVS transcript) is listed with respect to NM_016373.4.</p>

In [6]:
genome = 'hg38'
default_genotype = 'compound heterozygous'
WWOX_transcript='NM_016373.4'
wwox_id = "HGNC:12799"
wwox_symbol = "WWOX"
vvalidator = VariantValidator(genome_build=genome, transcript=WWOX_transcript)
all_variants = set()
variant_d = {}
patient_id_to_hgvs1_d = {}
patient_id_to_hgvs2_d = {}
for idx, row in df.iterrows():
    patient_id = str(row['patient_id'])
    v1 = row['DNA level (NM_016373.2)']
    v2 = row['Second mutation']
    all_variants.add(v1)
    all_variants.add(v2)
    patient_id_to_hgvs1_d[patient_id] = v1
    patient_id_to_hgvs2_d[patient_id] = v2
# The following are exon/multiexon deletions that cannot easily be interpreted by variant validator
# we will encode as structural since they are almost certainly loss of function
structural_vars = {
    "c.517-?_605+?del",
    "c.517-?_1056+ ?del",
    "c.-366-? *871+?del"
}
for v in all_variants:
    print(f"Encoding {v}")
    if v in structural_vars:
        var = StructuralVariant.chromosomal_deletion(cell_contents=v, gene_symbol=wwox_symbol, gene_id=wwox_id)
    else:
        var = vvalidator.encode_hgvs(v)
    variant_d[v] = var
print(f"Extracted {len(variant_d)} unique variants")

Encoding c.140C>G
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_016373.4%3Ac.140C>G/NM_016373.4?content-type=application%2Fjson
Encoding c.1005G>A
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_016373.4%3Ac.1005G>A/NM_016373.4?content-type=application%2Fjson
Encoding c.366_516del
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_016373.4%3Ac.366_516del/NM_016373.4?content-type=application%2Fjson
Encoding c.45_48delGGAC
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_016373.4%3Ac.45_48delGGAC/NM_016373.4?content-type=application%2Fjson
Encoding c.-366-? *871+?del
Encoding c.517-?_605+?del
Encoding c.517-?_1056+ ?del
Encoding c.889A>T
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_016373.4%3Ac.889A>T/NM_016373.4?content-type=application%2Fjson
Extracted 8 unique variants


<h1>Demographic data</h1>
<p>pyphetools can be used to capture information about age, sex, and individual identifiers. This information is stored in a map of "IndividualMapper" objects. Special treatment may be required for the indifiers, which may be used as the column names or row index.</p>

In [7]:
ageMapper = AgeColumnMapper.by_year('Age')
ageMapper.preview_column(df['Age'])

,original column contents,age
0,0,P0Y


In [8]:
sexMapper = SexColumnMapper(male_symbol='M', female_symbol='F', column_name='Sex')
sexMapper.preview_column(df['Sex'])

,original column contents,sex
0,F,FEMALE
1,F,FEMALE
2,M,MALE
3,F,FEMALE
4,F,FEMALE


In [9]:
encoder = CohortEncoder(df=df, 
                        hpo_cr=hpo_cr, 
                        column_mapper_d=column_mapper_d, 
                        individual_column_name="index", 
                        agemapper=ageMapper, 
                        sexmapper=sexMapper,
                        metadata=metadata,
                        pmid=PMID)
encoder.set_disease(disease_id='OMIM:616211', label='Developmental and epileptic encephalopathy 28')

In [10]:
individuals = encoder.get_individuals()

<h2>Variants</h2>
<p>Add the variants for each individual - it is hard to automate this with two alleles.</p>

In [11]:
for i in individuals:
    v1 = patient_id_to_hgvs1_d.get(i.id)
    v2 = patient_id_to_hgvs2_d.get(i.id)
    print(f"v1 {v1} v2 {v2}")
    if v1 == v2:
        var = variant_d.get(v1)
        var.set_homozygous()
        i.add_variant(var)
    else:
        var1 = variant_d.get(v1)
        var1.set_heterozygous()
        i.add_variant(var1)
        var2 = variant_d.get(v2)
        var2.set_heterozygous()
        i.add_variant(var2)

v1 c.366_516del v2 c.517-?_1056+ ?del
v1 c.1005G>A v2 c.517-?_605+?del
v1 c.45_48delGGAC v2 c.140C>G
v1 c.45_48delGGAC v2 c.140C>G
v1 c.889A>T v2 c.-366-? *871+?del


In [12]:
i1 = individuals[0]
phenopacket1 = i1.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh())
json_string = MessageToJson(phenopacket1)
print(json_string)

{
  "id": "PMID_25411445_1",
  "subject": {
    "id": "1",
    "timeAtLastEncounter": {
      "age": {
        "iso8601duration": "P0Y"
      }
    },
    "sex": "FEMALE"
  },
  "phenotypicFeatures": [
    {
      "type": {
        "id": "HP:0000252",
        "label": "Microcephaly"
      }
    },
    {
      "type": {
        "id": "HP:0012736",
        "label": "Profound global developmental delay"
      }
    },
    {
      "type": {
        "id": "HP:0008936",
        "label": "Axial hypotonia"
      }
    },
    {
      "type": {
        "id": "HP:0001251",
        "label": "Ataxia"
      },
      "excluded": true
    },
    {
      "type": {
        "id": "HP:0001257",
        "label": "Spasticity"
      },
      "excluded": true
    },
    {
      "type": {
        "id": "HP:0002375",
        "label": "Hypokinesia"
      }
    },
    {
      "type": {
        "id": "HP:0002063",
        "label": "Rigidity"
      }
    },
    {
      "type": {
        "id": "HP:0033725",
        

In [13]:
from IPython.display import display, HTML
ppacket_list = [i.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh()) for i in individuals]
table = PhenopacketTable(phenopacket_list=ppacket_list)
display(HTML(table.to_html()))

In [14]:
output_directory = "phenopackets"
Individual.output_individuals_as_phenopackets(individual_list=individuals,
                                             pmid=PMID,
                                             metadata=metadata.to_ga4gh(),
                                             outdir=output_directory)

We output 5 GA4GH phenopackets to the directory phenopackets
